In [1]:
!python -V

Python 3.9.7


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/home/abhishek-wsl/codes/MLops_project/final_codes/mlruns/2', creation_time=1693662607078, experiment_id='2', last_update_time=1693662607078, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    # df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    # df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
df_train = read_dataframe('/home/abhishek-wsl/codes/MLops_project/data/green_tripdata_2022-08.parquet')
df_val = read_dataframe('/home/abhishek-wsl/codes/MLops_project/data/test_data/green_tripdata_2022-01.parquet')

In [9]:
len(df_train), len(df_val)

(62354, 59603)

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [11]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

6.755884254920784

In [14]:
# with open('models/lin_reg.bin', 'wb') as f_out:
#     pickle.dump((dv, lr), f_out)

In [15]:
# with mlflow.start_run():

#     mlflow.set_tag("developer", "cristian")

#     mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
#     mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

#     alpha = 0.1
#     mlflow.log_param("alpha", alpha)
#     lr = Lasso(alpha)
#     lr.fit(X_train, y_train)

#     y_pred = lr.predict(X_val)
#     rmse = mean_squared_error(y_val, y_pred, squared=False)
#     mlflow.log_metric("rmse", rmse)

#     mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [16]:
import xgboost as xgb

In [17]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [18]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [19]:
# def objective(params):
#     with mlflow.start_run():
#         mlflow.set_tag("model", "xgboost")
#         mlflow.log_params(params)
#         booster = xgb.train(
#             params=params,
#             dtrain=train,
#             num_boost_round=100,
#             evals=[(valid, 'validation')],
#             early_stopping_rounds=50
#         )
#         y_pred = booster.predict(valid)
#         rmse = mean_squared_error(y_val, y_pred, squared=False)
#         mlflow.log_metric("rmse", rmse)

#     return {'loss': rmse, 'status': STATUS_OK}

In [20]:
# search_space = {
#     'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
#     'learning_rate': hp.loguniform('learning_rate', -3, 0),
#     'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
#     'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
#     'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
#     'objective': 'reg:linear',
#     'seed': 42
# }

# best_result = fmin(
#     fn=objective,
#     space=search_space,
#     algo=tpe.suggest,
#     max_evals=50,
#     trials=Trials()
# )

In [21]:
# mlflow.xgboost.autolog(disable=True)

In [22]:

best_params = {
    'learning_rate': 0.09585355369315604,
    'max_depth': 30,
    'min_child_weight': 1.060597050922164,
    'objective': 'reg:linear',
    'reg_alpha': 0.018060244040060163,
    'reg_lambda': 0.011658731377413597,
    'seed': 42
}

mlflow.xgboost.autolog()

booster = xgb.train(
    params=best_params,
    dtrain=train,
    num_boost_round=100,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)

#     y_pred = booster.predict(valid)
#     rmse = mean_squared_error(y_val, y_pred, squared=False)
#     mlflow.log_metric("rmse", rmse)

#     with open("models/preprocessor.b", "wb") as f_out:
#         pickle.dump(dv, f_out)
#     mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

#     mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

2023/09/02 20:59:13 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'e2aa3b69f91141c28eb04345bb516167', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[20:59:15] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:15.12152
[1]	validation-rmse:13.85131
[2]	validation-rmse:12.72272
[3]	validation-rmse:11.72649
[4]	validation-rmse:10.84843
[5]	validation-rmse:10.07535
[6]	validation-rmse:9.40456
[7]	validation-rmse:8.81798
[8]	validation-rmse:8.31803
[9]	validation-rmse:7.88105
[10]	validation-rmse:7.51017
[11]	validation-rmse:7.19789
[12]	validation-rmse:6.93361
[13]	validation-rmse:6.71262
[14]	validation-rmse:6.52675
[15]	validation-rmse:6.37785
[16]	validation-rmse:6.25193
[17]	validation-rmse:6.14694
[18]	validation-rmse:6.06116
[19]	validation-rmse:5.99266
[20]	validation-rmse:5.93868
[21]	validation-rmse:5.89346
[22]	validation-rmse:5.85533
[23]	validation-rmse:5.82495
[24]	validation-rmse:5.80217
[25]	validation-rmse:5.78359
[26]	validation-rmse:5.76846
[27]	validation-rmse:5.75516
[28]	validation-rmse:5.74673
[29]	validation-rmse:5.73740
[30]	validation

2023/09/02 20:59:27 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.


In [23]:
# with mlflow.start_run():
    
#     train = xgb.DMatrix(X_train, label=y_train)
#     valid = xgb.DMatrix(X_val, label=y_val)

#     best_params = {
#         'learning_rate': 0.09585355369315604,
#         'max_depth': 30,
#         'min_child_weight': 1.060597050922164,
#         'objective': 'reg:linear',
#         'reg_alpha': 0.018060244040060163,
#         'reg_lambda': 0.011658731377413597,
#         'seed': 42
#     }

#     mlflow.log_params(best_params)

#     booster = xgb.train(
#         params=best_params,
#         dtrain=train,
#         num_boost_round=1000,
#         evals=[(valid, 'validation')],
#         early_stopping_rounds=50
#     )

#     y_pred = booster.predict(valid)
#     rmse = mean_squared_error(y_val, y_pred, squared=False)
#     mlflow.log_metric("rmse", rmse)

#     with open("models/preprocessor.b", "wb") as f_out:
#         pickle.dump(dv, f_out)
#     mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

#     mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

In [24]:
# from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
# from sklearn.svm import LinearSVR

# mlflow.sklearn.autolog()

# for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

#     with mlflow.start_run():

#         mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
#         mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
#         mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

#         mlmodel = model_class()
#         mlmodel.fit(X_train, y_train)

#         y_pred = mlmodel.predict(X_val)
#         rmse = mean_squared_error(y_val, y_pred, squared=False)
#         mlflow.log_metric("rmse", rmse)
        